In [ ]:
import os
import glob
import numpy as np
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
import matplotlib.cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature

clon=180

# problem - plots land everywhere..
land_10m = cfeature.NaturalEarthFeature(
    "physical", "land", "10m", facecolor=cfeature.COLORS["land"]
)

def plot_decor(ax):
    ax.coastlines("10m")
    # ax.add_feature(ocean_10m, zorder=0)
    ax.add_feature(land_10m, zorder=-1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

In [ ]:
data_dir = 'data/202107070000_Taranaki_high'
fname = '21070700_Taranaki_12.0_0.01.nc'
files = glob.glob('{}/**/{}'.format(data_dir,fname), recursive=True)
files.sort()

In [ ]:
almost_zero=1.0e-6
threshold = 0.01
vname = 'prob_above_{}'.format(threshold)

ds = xr.open_mfdataset(files, combine='nested', concat_dim=['nwp'])
nmod, nt, ny, nx = ds['total_deposition'].shape

da_sum = np.zeros((nt, nx, ny))
for it in range(nt):
    for n in range(nmod):
        da1 = ds['total_deposition'].isel(time=it, nwp=n)
        da1m = np.zeros((nx,ny))
        da1m[da1>threshold] = 1
        da_sum[it] += da1m

    da_sum[it] = da_sum[it]/nmod
    da_sum[it] = np.where(da_sum[it]<=almost_zero, np.nan, da_sum[it])

ds_final = xr.Dataset({vname: (['time', 'lat', 'lon'], da_sum)}, coords={'lon': ds.lon, 'lat': ds.lat, 'time': ds.time})
ds_final

In [ ]:
from matplotlib.colorbar import ColorbarBase

max_hour = 10
cmap = mpl.cm.viridis
its = np.arange(0, max_hour)
norm = matplotlib.colors.Normalize(vmin=0, vmax=len(its))

fig, ax = plt.subplots(1,1, figsize=(12,8), subplot_kw={"projection": ccrs.PlateCarree(clon)})
plot_decor(ax=ax)

cs_list = []
for i,it in enumerate(its):
    da1 = ds_final[vname].isel(time=it)     
    ts = pd.to_datetime(str(da1['time'].data))     
    cs = da1.plot.contour(ax=ax, levels=[0.5], colors=[cmap(norm(i))], ls='--', transform=ccrs.PlateCarree())    
    cs_list += cs.collections
    
bounds = np.arange(max_hour + 1) - 0.5
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             orientation='vertical',
             label='Time since eruption [h]',
             ticks=its)
_ = ax.set_title(None)

In [ ]:
fig.savefig('plots/time_contours.png', bbox_inches='tight', dpi=200)